In [10]:

#%pip install mediapipe==0.9.0.1 
%pip install -q mediapipe==0.10.1


Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install ipywidgets ## for using tqdm 

In [12]:
#%pip install wget


In [13]:
#%wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [2]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import cv2

import numpy as np
import pandas as pd
import os
import shutil
from tqdm.notebook import tqdm

In [15]:
%pip install pyarrow fastparquet

Note: you may need to restart the kernel to use updated packages.


In [11]:
# code to generate dataset
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2,running_mode=VisionRunningMode.VIDEO)



#mp_holistic = mp.solutions.holistic
vid_dir  =  './AUTSL_dataset/train'
keyP_dir = './AUTSL_dataset/train_keypoints'
for i, filename in enumerate(tqdm(os.listdir(vid_dir))):
	if filename != "treated":
		old_file = f"{vid_dir}/{filename}"
		destination = f'{vid_dir}/treated/'
		if  os.path.exists(destination+filename):
			
			continue
		else:
			print(destination+filename)
			shutil.copy(old_file, destination)	

	

	cap = cv2.VideoCapture(f"{vid_dir}/treated/{filename}")
	detector = vision.HandLandmarker.create_from_options(options)
	#fps = cap.get(cv2.CAP_PROP_FPS)
	#holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.1)



	video_df = []
	frame_no=0
	data = []
	timestamp_ms = 0

		#fy = height/width
	for i in range(21):
			data.append({
					'type': 'left_hand',
					'landmark_index': i,
					'x': np.nan,
					'y': np.nan,
					'z': np.nan,
				})
	for i in range(21,42):
			data.append({
					'type': 'right_hand',
					'landmark_index': i,
					'x': np.nan,
					'y': np.nan,
					'z': np.nan,
				})

	while cap.isOpened():
		#print('\r',frame_no,end='')
		success, frame = cap.read()

		if not success: break
		# Calculate the timestamp for the current frame in millisecond
		timestamp_ms = int(cap.get(cv2.CAP_PROP_POS_MSEC))
		frame = cv2.resize(frame, (224,224))
		height,width,_ = frame.shape

		#print('\n',timestamp_ms)
		#print(frame.shape)
		#frame.flags.writeable = False
		frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
		#result = holistic.process(frame_rgb)
		mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
		result = detector.detect_for_video(mp_image,timestamp_ms)
		#print(result)



		hand_landmarks_list = result.hand_landmarks
		handedness_list = result.handedness
		# Loop through the detected hands
		i = 0
		for idx in range(len(hand_landmarks_list)):
			hand_landmarks = hand_landmarks_list[idx]
			handedness = handedness_list[idx]

			for landmark in hand_landmarks:  #
				data[i]['x'] = landmark.x
				data[i]['y'] = landmark.y
				data[i]['z'] = landmark.z
				#data[i]['confidence'] = landmark.visibility
				i+=1
				#print(data)
			#for categories in handedness:
				#print (landmark.x,landmark.y)


		frame_df = pd.DataFrame(data)
		frame_df.loc[:,'frame'] =  frame_no
		frame_df.loc[:, 'height'] = height/width
		frame_df.loc[:, 'width'] = width/width
		video_df.append(frame_df)
		

		#=========================
		frame_no +=1
	
	if frame_no != 0:
	
		video_df = pd.concat(video_df)
		#print(video_df)
		#holistic.close()
		video_df.to_parquet( f"{keyP_dir}/{filename}.to_parquet")
		old_file = f"{vid_dir}/{filename}"
		os.remove(old_file)
	


  0%|          | 0/11 [00:00<?, ?it/s]